In [9]:
import pandas as pd
import requests
import os
from nba_api.stats.static import teams
import io

def get_dates(start_year=2025, end_year=2026):
    dates = []
    for year in range(start_year, end_year):
        for team in teams.get_teams():
            team_id = team['id']
            path = f'../team/{year}ps/{team_id}.csv'
            if os.path.exists(path):
                df = pd.read_csv(path)
                if {'PLAYER_ID', 'HTM', 'VTM', 'GAME_DATE', 'GAME_ID'}.issubset(df.columns):
                    df = df[['PLAYER_ID', 'HTM', 'VTM', 'GAME_DATE', 'GAME_ID']]
                    df['year'] = year
                    df.drop_duplicates(inplace=True)
                    dates.append(df)
    if dates:
        return pd.concat(dates).drop_duplicates(subset='GAME_ID')
    else:
        return pd.DataFrame()

def fetch_game_csvs(dateframe, save_dir='game_data'):
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    all_game_data = []
    for _, row in dateframe.iterrows():
        year = int(row['year'])  # Use provided `year` column instead of parsing GAME_DATE
        game_id = row['GAME_ID']
        url = f'https://raw.githubusercontent.com/gabriel1200/player_sheets/refs/heads/master/game_report/{year}/{game_id}.csv'
        print(url)
        
        try:
            response = requests.get(url)
            response.raise_for_status()
            df = pd.read_csv(io.StringIO(response.text))
            df['GAME_ID'] = game_id
            df['date'] = row['GAME_DATE']
            df['HTM'] = row['HTM']
            df['VTM'] = row['VTM']
            df['year'] = year
            all_game_data.append(df)

            # Save raw CSV locally
            with open(os.path.join(save_dir, f'{year}_{game_id}.csv'), 'w', encoding='utf-8') as f:
                f.write(response.text)

        except requests.HTTPError as e:
            print(f'Failed to fetch {url}: {e}')
        except Exception as e:
            print(f'Unexpected error for {game_id}: {e}')
    
    return pd.concat(all_game_data, ignore_index=True) if all_game_data else pd.DataFrame()

def process_and_save_series_data(df, dateframe):
    # Merge HTM, VTM, GAME_DATE, year from dateframe
    for col in dateframe.columns:
        print(col)
    for col in df.columns:
        print(col)

    df.rename(columns={'GAME_DATE': 'date'}, inplace=True)

    # Home/away split using merged HTM/VTM
    home = df[df.HTM == df.TEAM_ABBREVIATION].copy()
    home.drop(columns='HTM', inplace=True)
    home.rename(columns={'VTM': 'opp_team'}, inplace=True)

    away = df[df.VTM == df.TEAM_ABBREVIATION].copy()
    away.drop(columns='VTM', inplace=True)
    away.rename(columns={'HTM': 'opp_team'}, inplace=True)

    none = df[df.HTM.isna()].copy().reset_index(drop=True)
    df = pd.concat([home, away, none], ignore_index=True)

    # Map opp_team from date/team_id
    oppframe = df[['TEAM_ID', 'date', 'opp_team']].dropna(subset=['opp_team']).drop_duplicates()
    df.drop(columns='opp_team', inplace=True)
    df = df.merge(oppframe, on=['TEAM_ID', 'date'], how='left')

    df['team'] = df['TEAM_ABBREVIATION']
    teammap = dict(zip(df['TEAM_ABBREVIATION'], df['TEAM_ID']))

    # Build current player index
    player_index = df[['PLAYER_NAME', 'PLAYER_ID', 'team', 'TEAM_ID', 'opp_team', 'year']].copy()
    player_index['opp_id'] = player_index['opp_team'].map(teammap)
    player_index.drop_duplicates(inplace=True)

    # Merge with existing index file if it exists
    index_path = 'series_index_players.csv'
    if os.path.exists(index_path):
        existing_index = pd.read_csv(index_path)
        combined = pd.concat([existing_index, player_index], ignore_index=True)
        combined.drop_duplicates(
            subset=['PLAYER_ID', 'team', 'TEAM_ID', 'opp_team', 'year'], keep='last', inplace=True
        )
        combined.to_csv(index_path, index=False)
    else:
        player_index.to_csv(index_path, index=False)

    # Continue data cleanup
    df = df.dropna(subset=['opp_team'])
    df['opp_id'] = df['opp_team'].map(teammap)
    df.sort_values(by='date', inplace=True)
    df['series_key'] = df['team'] + '_' + df['opp_team'] + '_' + df['year'].astype(str)

    df.to_csv('playoff_data.csv', index=False)

    # Save series-specific dataframes
    series_dir = '../series/data'
    os.makedirs(series_dir, exist_ok=True)
    for key, group in df.groupby('series_key'):
        safe_key = key.replace('/', '-').replace('\\', '-')  # sanitize for filesystem
        group.to_csv(os.path.join(series_dir, f'{safe_key}.csv'), index=False)

    return df



# === Run Full Pipeline ===
start_year = 2025
end_year = 2026

dates = get_dates(start_year, end_year)
if dates.empty:
    print(f'No game date data found for {start_year}-{end_year - 1}.')
else:
    raw_df = fetch_game_csvs(dates)
    if raw_df.empty:
        print('No game CSVs successfully fetched.')
    else:
        processed_df = process_and_save_series_data(raw_df, dates)




https://raw.githubusercontent.com/gabriel1200/player_sheets/refs/heads/master/game_report/2025/42400111.csv
https://raw.githubusercontent.com/gabriel1200/player_sheets/refs/heads/master/game_report/2025/42400112.csv
https://raw.githubusercontent.com/gabriel1200/player_sheets/refs/heads/master/game_report/2025/42400113.csv
https://raw.githubusercontent.com/gabriel1200/player_sheets/refs/heads/master/game_report/2025/42400114.csv
https://raw.githubusercontent.com/gabriel1200/player_sheets/refs/heads/master/game_report/2025/42400115.csv
https://raw.githubusercontent.com/gabriel1200/player_sheets/refs/heads/master/game_report/2025/42400211.csv
https://raw.githubusercontent.com/gabriel1200/player_sheets/refs/heads/master/game_report/2025/42400212.csv
https://raw.githubusercontent.com/gabriel1200/player_sheets/refs/heads/master/game_report/2025/42400213.csv
https://raw.githubusercontent.com/gabriel1200/player_sheets/refs/heads/master/game_report/2025/42400214.csv
https://raw.githubuserconten